In [7]:
import numpy as np
import pandas as pd
import utils
from neuron_explainer.activations.activations import ActivationRecordSliceParams, load_neuron

In [8]:
EXPLAINER_MODEL = "gpt-3.5-turbo"
INPUT_PATH = "inputs/test_neurons.csv"

neuron_df = pd.read_csv(INPUT_PATH)
neuron_df

,layer,neuron,score,explanation
0,0,286,0.663509,words related to comparison.
1,10,1838,0.676886,phrases describing positions or situations in...
2,20,193,0.455764,verbs indicating questioning or challenging be...
3,30,1685,0.537097,expressions of gratitude and agreeing to rece...
4,40,431,0.365750,"numbers related to time, dates, and measureme..."


In [9]:
modes = ["Original", "Summary", "Highlight", "HighlightSummary", "AVHS"]
new_results = {"layer":[], "neuron":[], "explainer":[]}
for mode in modes:
    new_results[mode] = []

for i, row in neuron_df.iterrows():
    layer = row["layer"]
    neuron = row["neuron"]
    new_results["layer"].append(layer)
    new_results["neuron"].append(neuron)
    new_results["explainer"].append(EXPLAINER_MODEL)
    neuron_record = load_neuron(layer, neuron)
    
    for mode in modes:
        token_results = await utils.get_explanation(mode=mode, neuron_record=neuron_record, explainer_model=EXPLAINER_MODEL, get_token_only = True)
        new_results[mode].append(token_results['total_tokens'])

new_df = pd.DataFrame(new_results)
new_df

,layer,neuron,explainer,Original,Summary,Highlight,HighlightSummary,AVHS
0,0,286,gpt-3.5-turbo,2353,932,866,979,1132
1,10,1838,gpt-3.5-turbo,2378,950,885,1009,1168
2,20,193,gpt-3.5-turbo,2399,969,889,1021,1181
3,30,1685,gpt-3.5-turbo,2265,994,901,1076,1588
4,40,431,gpt-3.5-turbo,2488,971,884,1027,1279


In [12]:
new_df.to_csv("test_results/"+INPUT_PATH.split("/")[-1].split(".")[0] + "_tokens.csv")

In [11]:
for mode in modes:
    avg = np.mean(new_df[mode])
    print(mode, avg)

Original 2376.6
Summary 963.2
Highlight 885.0
HighlightSummary 1022.4
AVHS 1269.6
